In [3]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import statistics
import kmax as kx
import utility as util
from hierarchy import *
from scipy.optimize import fsolve
from IPython.core.debugger import set_trace
import scipy.stats as stats
import pylab
import statsmodels.api as sm

# Simulations 3

## Goals:
- Find path segment length for a typical vertex

In [4]:
# Calculate the length of the path passing 
# through `node` in the optimal path cover
def path_seg_length(tree, node):
    path,_,_ = util.path_cover(tree)
    path_subgraph = tree.subgraph(path)

    return len(nx.node_connected_component(path_subgraph, node))

In [32]:
sample_tree = nx.Graph(GWBP(0.9, MAXLEVEL=100))
for u,v in sample_tree.edges():
    sample_tree[u][v]['weight'] = 1

#print(path_seg_length(sample_tree, 0))

[0, 1, 2, 3]
